# Storage toolbox

There are storage soft limits present in the CodeBook.

Each time a user starts the CodeBook it takes some time to synchronise the user files with the CodeBook server.
If the user has too many files then the CodeBook can hit start-up time limit and will fail to start.
In such a case the user will need to contact support to compress the stored files.

To prevent start-up failure we notify users when they are close to the limit.

_Note:_ Use `__TEMP__` folder to store bulk files during your session to not exceed storage soft limits.

## This notebook contains:
1. Helper scripts to check if the storage limits are reached.
2. Helper scripts to manipulate user files (compress and delete).

## 1.1. Check if the limits are reached
The current soft limits are:
- Total size of user's files: 2GB
- Number of user's files: 2000

In [ ]:
from importlib import import_module as im

STORAGE_LIMIT_SIZE_GB = 2  # Total size - soft limit
STORAGE_LIMIT_COUNT = 2000  # Number of files - soft limit

popen = im("os").popen
total_size = popen("du -sh ~ | awk '{{print $1}}'").read().strip()
if total_size.endswith("G"):
    if float(total_size.rstrip("G")) > STORAGE_LIMIT_SIZE_GB:
        print(f"The SIZE soft LIMIT ({STORAGE_LIMIT_SIZE_GB}G) IS REACHED!")
print(f"Total size of the files in the home folder: {total_size}\n")

total_count = int(popen("find ~ -type f -not -path '*/.*' | wc -l").read().strip())
if total_count > STORAGE_LIMIT_COUNT:
    print(f"The NUMBER OF FILES soft LIMIT ({STORAGE_LIMIT_COUNT}) IS REACHED!")
print(f"Total number of the files in the home folder: {total_count}")

## 1.2. Show size per folder

In [ ]:
from importlib import import_module as im

popen = im("os").popen
folders = popen("find ~ -type d -not -path '*/.*' -exec du -ch --exclude '*/.*' {} +").read()
print(folders.replace("/home/jovyan", "~"))

## 1.3. Show number of files per folder

In [ ]:
from importlib import import_module as im

popen = im("os").popen
folders = popen('/usr/bin/bash -c \'find ~ -type d -not -path "*/.*" -print0 | while read -d "" -r dir; do files=("$dir"/*); printf "%5d files in folder %s\n" "${#files[@]}" "$dir"; done\'').read()
print(folders.replace("/home/jovyan", "~"))

## 2.1. Compress a given folder
If you have a folder with many files, you may use this snippet to compress it into an archive to download from the CodeBook environment easily.

In [ ]:
# This cell takes two parameters: FOLDER_TO_COMPRESS and ARCHIVE_NAME
#  and creates an archive ARCHIVE_NAME.zip in the home folder.

# After the archive is created, you can download it to the local machine, 
#  then delete the compressed folder and the archive from the CodeBook environment.

# The path is relative to the user home folder and should start with ~/
# Example: "~/folder/to/compress"
FOLDER_TO_COMPRESS = "~/folder/to/compress"

# Extension .zip will be added to the filename.
ARCHIVE_NAME = "archive_name"


from importlib import import_module as im
from pathlib import Path

print("Compressing...")
ma = im("shutil").make_archive
folder_path = Path(
    FOLDER_TO_COMPRESS if FOLDER_TO_COMPRESS.startswith("~/") else "~/" + FOLDER_TO_COMPRESS
).expanduser()
archive_path = Path("~/"+ARCHIVE_NAME).expanduser()
archive = ma(archive_path, "zip", folder_path).replace("/home/jovyan", "~")
print(f"The archive is created: {archive}")

## 2.2. Delete a given folder
Use this snippet to delete non-empty folders.
### Important: Deleted folder couldn't be restored!

In [ ]:
# This cell takes one parameter: FOLDER_TO_DELETE

# The path is relative to the user home folder and should start with ~/
# Example: "~/folder/to/delete"
FOLDER_TO_DELETE = "~/folder/to/delete"


from importlib import import_module as im

popen = im("os").popen
folder_path = FOLDER_TO_DELETE if FOLDER_TO_DELETE.startswith("~/") else "~/" + FOLDER_TO_DELETE
confirmation = input(f'You are going to PERMANENTLY delete folder: {folder_path}.\nType "yes" to proceed: ')
if confirmation.lower() == "yes":
    print("Deleting...")
    failure = popen(f"rm -rfv {folder_path}").read()
    print("Done.")
else:
    print("Deletion canceled.")